In [3]:
# hmmlearn使用HMM

In [ ]:
# hmmlearn实现了三种HMM模型类，
# 按照观测状态是连续状态还是离散状态，可以分为两类。
# GaussianHMM和GMMHMM是连续观测状态的HMM模型，
# 而MultinomialHMM是离散观测状态的模型--是我们学习的时候使用的模型。

In [4]:
# 对于MultinomialHMM的模型，使用比较简单，
# 'startprob_'参数对应我们的隐藏状态初始分布Π，
# 'transmat_'对应我们的状态转移矩阵A，
# 'emissionprob_'对应我们的观测状态概率矩阵B。

In [5]:
# 对于连续观测状态的HMM模型，GaussianHMM类假设观测状态符合高斯分布，
# 而GMMHMM类则假设观测状态符合混合高斯分布。

# 一般情况下，我们使用GussianHMM即高斯分布的观测状态即可。
# 以下对于连续观测状态的HMM模型，我们只讨论GaussianHMM类。

In [6]:
# 在GaussianHMM类中，
# "startprob_"参数对应我们的隐藏状态初始分布Π, 
# "transmat_"对应我们的状态转移矩阵A, 
# 比较特殊的是观测状态概率的表示方法，
# 此时由于观测状态是连续值，我们无法像MultinomialHMM一样直接给出矩阵B。
# 而是采用给出各个隐藏状态对应的观测状态高斯分布的概率密度函数的参数。

In [ ]:
# 如果观测序列是一维的，则观测状态的概率密度函数是一维的普通高斯分布。
# 如果观测序列是N维的，则隐藏状态对应的观测状态的概率密度函数是N维高斯分布。
# 高斯分布的概率密度函数参数可以用μ表示高斯分布的期望向量，Σ表示高斯分布的协方差矩阵。
# 在GaussianHMM类中，
# “means”用来表示各个隐藏状态对应的高斯分布期望向量μ形成的矩阵，
# 而“covars”用来表示各个隐藏状态对应的高斯分布协方差矩阵Σ形成的三维张量。

In [7]:
# MultinomialHMM实例

In [8]:
import numpy as np
from hmmlearn import hmm

In [9]:
# 隐藏状态集合
states = ['box 1','box 2','box 3']
n_states = len(states)

In [10]:
# 可见观察符合集合
observations = ['red','white']
n_observations = len(observations)

In [15]:
# 初始状态π
start_probability = np.array([0.2,0.4,0.4])

In [12]:
# 状态转移概率矩阵A
transition_probability = np.array([[0.5,0.2,0.3],
                                  [0.3,0.5,0.2],
                                  [0.2,0.3,0.5]])

In [13]:
# 符合发射概率矩阵B
emission_probability = np.array([[0.5,0.5],
                                [0.4,0.6],
                                [0.7,0.3]])

In [20]:
# 构建HMM模型
model = hmm.MultinomialHMM(n_components = n_states)
model.startprob_ = start_probability
model.transmat_ = transition_probability
model.emissionprob_ = emission_probability

In [21]:
# 跑一跑HMM基本问题2：维特比算法的解码过程 P(S|O)

In [22]:
# 定义观察序列
seen = np.array([[0,1,0]]).T
seen.shape

(3, 1)

In [39]:
# 用维特比算法解码获得对应的状态序列
logprob, box = model.decode(seen, algorithm='viterbi')
print(logprob)
print(box)
print("The ball picked:", ", ".join(map(lambda x: observations[int(x)], seen)))
print('The hidden box:',','.join(map(lambda x: states[x], box)))


-4.219907785197447
[2 2 2]
The ball picked: red, white, red
The hidden box: box 3,box 3,box 3


In [40]:
# 也可以使用predict函数
box2 = model.predict(seen)
print(box2)
print("The ball picked:", ", ".join(map(lambda x: observations[int(x)], seen)))
print("The hidden box", ", ".join(map(lambda x: states[int(x)], box2)))

[2 2 2]
The ball picked: red, white, red
The hidden box box 3, box 3, box 3


In [41]:
# HMM问题一的求解P(O|λ)观测序列概率问题
print(model.score(seen))
# 要注意的是score函数返回的是以自然数为底的对数概率只。
# 手动计算的结果是对未取对数的原始概率值0.13022

-2.038545309915233


In [49]:
# 对比一下
import math
math.log(0.13022,math.e)

-2.038529951173421

In [50]:
# HMM问题三的求解，模型参数问题
# 由于鲍姆-韦尔奇算法是基于EM算法的近似算法，我们需要多跑几次。
# 下面我们跑三次，选择一个比较优的模型参数。

In [52]:
import numpy as np
from hmmlearn import hmm

In [53]:
# 定义隐藏状态集合和 可见观察集合
states = ["box 1", "box 2", "box3"]
n_states = len(states)

observations = ["red", "white"]
n_observations = len(observations)

In [54]:
# 定义模型
model2 = hmm.MultinomialHMM(n_components = n_states, 
                            n_iter = 20, 
                            tol = 0.01)

In [55]:
# 不同的可见的观察序列，用于模型的参数估计
X2 = np.array([[0,1,0,1],
              [0,0,0,1],
              [1,0,1,1]])

In [56]:
model2.fit(X2)

MultinomialHMM(algorithm='viterbi', init_params='ste', n_components=3,
        n_iter=20, params='ste',
        random_state=<mtrand.RandomState object at 0x1113f5cf0>,
        startprob_prior=1.0, tol=0.01, transmat_prior=1.0, verbose=False)

In [57]:
# 初始概率π
print(model2.startprob_)
# 状态转移概率矩阵A
print(model2.transmat_)
# 符号发射概率矩阵B
print(model2.emissionprob_)
# 基本问题一的P(O|λ)概率
print(model2.score(X2))

[2.09524164e-02 2.41818597e-23 9.79047584e-01]
[[3.00849648e-01 6.50639228e-01 4.85111240e-02]
 [2.15137764e-01 3.87834688e-01 3.97027547e-01]
 [9.97272196e-03 9.89910103e-01 1.17174628e-04]]
[[9.25220802e-01 7.47791984e-02]
 [1.34889296e-01 8.65110704e-01]
 [9.99171912e-01 8.28087714e-04]]
-6.68941903384501


In [58]:
model2.fit(X2)

MultinomialHMM(algorithm='viterbi', init_params='ste', n_components=3,
        n_iter=20, params='ste',
        random_state=<mtrand.RandomState object at 0x1113f5cf0>,
        startprob_prior=1.0, tol=0.01, transmat_prior=1.0, verbose=False)

In [59]:
# 初始概率π
print(model2.startprob_)
# 状态转移概率矩阵A
print(model2.transmat_)
# 符号发射概率矩阵B
print(model2.emissionprob_)
# 基本问题一的P(O|λ)概率
print(model2.score(X2))

[1.00000000e+00 2.58198790e-11 4.40625994e-22]
[[1.42431066e-07 2.22914196e-01 7.77085661e-01]
 [7.57072539e-01 1.31134370e-01 1.11793091e-01]
 [5.66587097e-01 1.00935330e-01 3.32477572e-01]]
[[9.99935344e-01 6.46563741e-05]
 [3.54134111e-01 6.45865889e-01]
 [1.06751340e-01 8.93248660e-01]]
-6.498981934979207


In [60]:
# ...结果这里就略去了，最终我们会选择分数最高的模型参数。

In [61]:
# 以上就是用MultinomialHMM解决HMM模型三个问题的方法。

In [62]:
print('*'*50)

**************************************************


In [63]:
# GaussianHMM实例
# 下面我们再给一个GaussianHMM的实例，
# 在这个实例汇总，我们的观测状态是二维的，而隐藏状态有4个。
# 因此我们'means'参数是4*2的矩阵，而'covars'参数是4*2*2的张量。

In [64]:
# 初始状态概率π
startprob = np.array([0.6,0.3,0.1,0.0])

In [66]:
# 隐藏序列状态转移概率矩阵A
transmat = np.array([[0.7,0.2,0.0,0.1],
                    [0.3,0.5,0.2,0.0],
                    [0.0,0.3,0.5,0.2],
                    [0.2,0.0,0.2,0.6]])

In [67]:
# The means of each compont
means = np.array([[0.0,0.0],
                 [0.0,11.0],
                 [9.0,10.0],
                 [11.0,-1.0]])

In [68]:
# The covariance of each component
covars = 0.5*np.tile(np.identity(2),(4,1,1))

In [69]:
# 创建HMM实例，设置参数
model3 = hmm.GaussianHMM(n_components = 4, covariance_type ='full')

In [70]:
# Instead of fitting it from the data, we directly set the estimated
# parameters, the means and covariance of the components
model3.startprob_ = startprob
model3.transmat_ = transmat
model3.means_ = means
model3.covars_ = covars

In [71]:
# 注意上面有个参数covariance_type，
# 取值为"full"意味所有的μ,Σ都需要指定。
# 取值为“spherical”则Σ的非对角线元素为0，对角线元素相同。
# 取值为“diag”则Σ的非对角线元素为0，对角线元素可以不同，
# "tied"指所有的隐藏状态对应的观测状态分布使用相同的协方差矩阵 Σ